In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1


import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridspecLayout


import matplotlib 
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
set_input_table = widgets.Text(description='Input Table:', value='content-insights-dev-249001.echostar_slingtv.customer_view_by_uniqueid', disabled=False)
set_save_table = widgets.Text(description='Save Table:', value='content-insights-dev-249001.ds.encostar_slingtv_outlier', disabled=False)
set_asset_id = widgets.Text(description='Asset ID:', value='asset_canonical_id', disabled=False)
set_viewer_id = widgets.Text(description='Viewer ID:', value='uniqueId', disabled=False)
set_contamination = widgets.FloatSlider(
    value=0.02,
    min=0.01,
    max=100,
    step=0.01,
    description='Outlier (%):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
set_date = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

In [3]:
grid = GridspecLayout(3, 2)
grid[0, 0] = set_input_table
grid[0, 1] = set_save_table
grid[1, 0] = set_asset_id
grid[1, 1] = set_viewer_id
grid[2, 0] = set_date
grid[2, 1] = set_contamination
grid

GridspecLayout(children=(Text(value='content-insights-dev-249001.echostar_slingtv.customer_view_by_uniqueid', …

In [25]:
asset_id = set_asset_id.value
viewer_id = set_viewer_id.value
date = str(set_date.value)
contamination = set_contamination.value
input_table = set_input_table.value
save_table = set_save_table.value

print("asset id: {0} \nviewer id: {1} \ndate: {2} \ncontamination:{3}% \ninput table: {4} \noutput table: {5}".format(asset_id, viewer_id, date, contamination, input_table, save_table))

asset id: asset_canonical_id 
viewer id: uniqueId 
date: 2020-05-14 
contamination:0.02% 
input table: content-insights-dev-249001.echostar_slingtv.customer_view_by_uniqueid 
output table: content-insights-dev-249001.ds.encostar_slingtv_outlier


#### connect BigQuery

In [26]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/adam/Documents/code/PycharmProjects/content-qa-dev-1.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/adam/Documents/code/PycharmProjects/content-qa-dev-1.json


In [27]:
client = bigquery.Client()
# Perform a query.
QUERY = (
    'SELECT {0} uniqueId, count(1) total_plays, sum(session_playTimeMs) / 1000 / 60 total_playMins, count(distinct {1} ) unique_assets FROM `{2}` WHERE session_date = \"{3}\" Group by 1'.format(viewer_id, asset_id, input_table, date)
)
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

In [28]:
rowsdf = rows.to_dataframe()
rowsdf.head()

,uniqueId,total_plays,total_playMins,unique_assets
0,dd4ce04c-efc7-11e5-8720-128e56ace683,59,1455.423233,57
1,bb58f8c8-63a3-11e5-b4c3-0a42a46b5c52,36,224.184650,11
2,ca4b9a82-c509-11e8-a38d-0ae1aa194538,50,1465.426217,46
3,c8d38494-50e5-11ea-91a7-12546226326b,42,1133.686733,39
4,9bcac7d2-872c-11ea-8262-0ea96829ae61,38,855.058750,20


#### detect outliers 

In [29]:
to_model_columns=rowsdf.columns[1:4]
to_model_columns

clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.02), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
clf.fit(rowsdf[to_model_columns])
pred = clf.predict(rowsdf[to_model_columns])
rowsdf['anomaly']=pred
outliers=rowsdf.loc[rowsdf['anomaly']==-1]
outlier_index=list(outliers.index)
#print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
print(rowsdf['anomaly'].value_counts())

 1    1444911
-1      29488
Name: anomaly, dtype: int64


In [30]:
rowsdf.head()

,uniqueId,total_plays,total_playMins,unique_assets,anomaly
0,dd4ce04c-efc7-11e5-8720-128e56ace683,59,1455.423233,57,-1
1,bb58f8c8-63a3-11e5-b4c3-0a42a46b5c52,36,224.184650,11,1
2,ca4b9a82-c509-11e8-a38d-0ae1aa194538,50,1465.426217,46,-1
3,c8d38494-50e5-11ea-91a7-12546226326b,42,1133.686733,39,1
4,9bcac7d2-872c-11ea-8262-0ea96829ae61,38,855.058750,20,1


In [31]:
outdf = rowsdf[rowsdf['anomaly'] == -1]
outdf['date'] = date
outdf.head()

,uniqueId,total_plays,total_playMins,unique_assets,anomaly,date
0,dd4ce04c-efc7-11e5-8720-128e56ace683,59,1455.423233,57,-1,2020-05-14
2,ca4b9a82-c509-11e8-a38d-0ae1aa194538,50,1465.426217,46,-1,2020-05-14
17,a6ff73ae-b6db-11e9-b952-129a161a09ae,230,522.584983,188,-1,2020-05-14
19,13f3b5ce-22e3-11ea-8c79-0a0ecd9f4ba3,76,560.000767,36,-1,2020-05-14
32,9aed4cac-1124-11e8-9e09-12fbc3de3046,58,1697.367033,44,-1,2020-05-14


In [32]:
insertdf = outdf[['uniqueId', 'total_plays', 'total_playMins', 'unique_assets', 'date']].reset_index(drop=True)
insertdf.head()

,uniqueId,total_plays,total_playMins,unique_assets,date
0,dd4ce04c-efc7-11e5-8720-128e56ace683,59,1455.423233,57,2020-05-14
1,ca4b9a82-c509-11e8-a38d-0ae1aa194538,50,1465.426217,46,2020-05-14
2,a6ff73ae-b6db-11e9-b952-129a161a09ae,230,522.584983,188,2020-05-14
3,13f3b5ce-22e3-11ea-8c79-0a0ecd9f4ba3,76,560.000767,36,2020-05-14
4,9aed4cac-1124-11e8-9e09-12fbc3de3046,58,1697.367033,44,2020-05-14


#### save BigQuery

In [33]:
loadbqtablereference = bigquery.table.TableReference.from_string(save_table)
client.load_table_from_dataframe(insertdf, loadbqtablereference)